# spaCyのpipeline周りに詳しくなる

https://spacy.io/usage/processing-pipelines

上から写経していく。

> When you call nlp on a text, spaCy first tokenizes the text to produce a Doc object. The Doc is then processed in several different steps – this is also referred to as the processing pipeline. The pipeline used by the default models consists of a tagger, a parser and an entity recognizer. Each pipeline component returns the processed Doc, which is then passed on to the next component.

要は `nlp` メソッドにテキストを渡すと、トークン化したテキストをDocクラスのオブジェクトに入れて返してくれると。
そのDocオブジェクトは pipeline という仕組みで、連鎖的に処理した結果をDocオブジェクトのバケツリレーをするってことかな。
pipeline には tagger、parser、entity recognizer(ner) があるよ。

なるほど。
docオブジェクトの型を見てみよう。

In [4]:
import spacy

nlp = spacy.load("en")
doc = nlp("This is a text")
type(doc)

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

enモデルが無いよ、って怒られました。

https://spacy.io/usage/models

QuickStartの通りにやってみる

In [5]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 476 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-py3-none-any.whl size=12011738 sha256=4e741a4ef6924b14806dc4789ff4156bf93b98c79d33f5959516f6a04c73f4bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-yazrb305/wheels/51/19/da/a3885266a3c241aff0ad2eb674ae058fd34a4870fef1c0a5a0
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


ダウンロードできた。
コードを実行してみる

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

むむ。
jupyter notebook上だとアカンのか？
一度Dockerfileに書いてビルドし直してみる。

ビルドし直してみた。
再度実行してみる。

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

何もエラー出ない。成功か。
docの型を見てみよう。

In [4]:
doc = nlp("This is a text")
type(doc)

spacy.tokens.doc.Doc

`spacy.tokens.doc.Doc` なるほど。
pipeline は何が設定されているか。

In [6]:
for p in nlp.pipeline:
    print(p)

('tagger', <spacy.pipeline.pipes.Tagger object at 0x7fc3c78613d0>)
('parser', <spacy.pipeline.pipes.DependencyParser object at 0x7fc39292ede0>)
('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x7fc3928c5360>)


ふむふむ。
tagger、parser、ner 確かに。

ちなみに、モデルのQuickStart見てたら、こんな書き方↓もできるみたい。

In [10]:
import en_core_web_sm # 文字列でloadするモデルを指定する方法の他に、モジュールとして読み込む方法があるようだ
nlp = en_core_web_sm.load() # 引数なしの load メソッドが nlp を返すのか
doc = nlp("This is a text")
print(doc)

for p in nlp.pipeline:
    print(p)

This is a text
('tagger', <spacy.pipeline.pipes.Tagger object at 0x7fc3903805d0>)
('parser', <spacy.pipeline.pipes.DependencyParser object at 0x7fc3928bad70>)
('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x7fc3928ba9f0>)


nlp オブジェクトってなんだ

In [11]:
type(nlp)

spacy.lang.en.English

ふーん

> The processing pipeline always depends on the statistical model and its capabilities. For example, a pipeline can only include an entity recognizer component if the model includes data to make predictions of entity labels. This is why each model will specify the pipeline to use in its meta data, as a simple list containing the component names:

処理パイプラインは、常に統計モデルとその機能に依存しています。 たとえば、モデルにエンティティラベルの予測を行うためのデータが含まれている場合、パイプラインにはエンティティ認識コンポーネントのみを含めることができます。 これが、各モデルがメタデータで使用するパイプラインを、コンポーネント名を含む単純なリストとして指定する理由です。